# Applied Data Science and Machine Intelligence
## A program by IITM and TalentSprint
### Assignment 11: PCA

## Learning Objectives

At the end of the experiment, you will be able to:

* Dimensionality reduction
* PCA

## Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook= "M2_Assignment_11_PCA_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.extras.talentsprint.com/CDS/Datasets/Real_estate.csv")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None
    
    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://adsmi.iitm.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None
  
  
# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None
  
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None
  

def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError 
    else: 
      return Answer
  except NameError:
    print ("Please answer Question")
    return None
  

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup() 
else:
  print ("Please complete Id and Password cells before running setup")



## Importing necessary packages for this assignment.
Lets first import the necessary packages required for this notebook.

In [ ]:
# Importing Standard Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Importing sklearn Libraries
from sklearn import datasets
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#### **Dimensionality Reduction**

In Data Science and Machine Learning applications, we encounter different types of data, sizes, structure, format and complexities. The point of discussion in this section is related to large volumes of data with a large number of features. The main reason for such large number of features is owed to the application, and the process of data acquisition.

In most applications, although there might be a large number of features, it might seem like more data gives better predictions.

Counterintuitively, some features might be 

* redundant, 
* having significant percent of missing observations
* uncorrelated to the target variable
* having low variance
* slowing down the ML model's learning capability

To overcome the above disadvantages, we perform dimensionality reduction.
Dimensionality in the context of Tabular data refers to the number of features of the dataset. There are various ways to perform Dimensionality reduction. Some of the commonly used techniques are as follows.


- Lasso Regression
- Principal Component Analysis
- Independent Component Analysis
- Forward Feature Selection
- Backward Feature Elimination
- Missing Value Ratio
- Random Forest
- Factor Models

But all the techniques cannot be applied to all the scenarios. For example, certain techniques work with linearly coorelated features only. So the applicability is case-by-case specific.

For more information, please refer to the links: 

[Link 1](https://www.researchgate.net/publication/342182637_A_Short_Survey_of_Dimensionality_Reduction_Techniques)

[Link 2](http://www.columbia.edu/~mh2078/QRM/DimensionReductionTechniques.pdf) 

[Link 3](https://www.cc.gatech.edu/~isbell/tutorials/dimred-survey.pdf)

#### **Principal Component Analysis (PCA)**

PCA is an unsupervised statistical technique used to examine
the interrelations among dataset's features in order to identify the underlying structure of those features.

PCA is the most sought-after Dimensionality reduction technique for linear. The reason is due to the fact that PCA has one of the best mean-square-error metrics. Under the hood, it employs Singular Value Decomposition popularly known as SVD. SVD is faster than other dimensionality reduction techniques, due to the Eigen-value Decomposition as the main part of its operation. PCA requires that the dataset must be standardized (zero mean and unit standard deviation)


PCA works as follows:

- The user inputs the required number of dimensions/PCs $n$.

- It seeks to reduce the dimension of the data by finding a few orthogonal linear combinations (the PCs) of the original variables with the largest variance. 

- The first PC, **$s_1$**, is the linear combination with
the largest variance. This is the most dominant PC, as it captures the maximum variance of the other features' linear combination.

- The second PC, **$s_2$** is the linear combination with the second largest variance and orthogonal to the first PC, **$s_1$**.

- The third PC, **$s_3$** is the linear combination with the third largest variance and orthogonal to the first and second PCs (**$s_1$** and **$s_2$**)

- and so on upto the $n^{th}$ PC **$s_n$**.


One major disadvantage is that the actual features are reduced to just unkown variables. Example: If you have a Vehicle dataset with Cost, Engine_capacity, Number_of_cylinders, Number_of_bolts_1, Number_of_bolts_2, ....., then after applying PCA it will become s1, s2, etc. This is useful from the ML perspective, but from the Business perspective, these features that act as drivers to key-decision making would be lost.



In the figure below, we see that 3 dimensions has been reduced to just 2.

More information [here](https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e)

![](https://miro.medium.com/max/1200/1*ba0XpZtJrgh7UpzWcIgZ1Q.jpeg)


####  Simple Regression base-model with train-test split

In [ ]:
from sklearn.datasets import load_breast_cancer
breast = load_breast_cancer()
breast_data = breast.data
breast_data.shape

In [ ]:
breast_labels = breast.target
breast_labels.shape

In [ ]:
import pandas as pd
import numpy as np

labels = np.reshape(breast_labels,(569,1))
final_breast_data = np.concatenate([breast_data,labels],axis=1)

features = list(breast.feature_names)
cols = features + ['label']
breast_dataset = pd.DataFrame(final_breast_data, columns=cols)

breast_dataset.sample(n=10)

In [ ]:
from sklearn.preprocessing import StandardScaler
x = breast_dataset.loc[:, features].values
x = StandardScaler().fit_transform(x) # normalizing the features
normalised_breast = pd.DataFrame(x,columns=features)
normalised_breast.head()

In [ ]:
from sklearn.decomposition import PCA

n_pc = 2
cols_pc = []
for nth_pc in range(n_pc):
  cols_pc.append(f"pc_{nth_pc+1}")

pca_breast = PCA(n_components=n_pc)
principalComponents_breast = pca_breast.fit_transform(x)

principal_breast_Df = pd.DataFrame(data = principalComponents_breast, columns = cols_pc)
principal_breast_Df.head()

In [ ]:
print('Explained variation per principal component: {}'.format(pca_breast.explained_variance_ratio_))


In [ ]:
principal_breast_Df['pc_2']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


sns.scatterplot(principal_breast_Df['pc_1'], principal_breast_Df['pc_2'], hue=breast_dataset['label'])



In [ ]:
principal_breast_Df['pc_1'][breast_dataset['label']]

## Please answer the questions below to complete the experiment:

Please find the eigen values using SVD transformation use numpy library \begin{bmatrix}
1 & 2 & 3 & 4 \\
5 & 6 & 7 & 8 \\
3 & 7 & 8 & 9
\end{bmatrix}




In [ ]:
#@title Select the correct options for eigen value { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["", "[20.07711544,  1.80274635,  0.81212135]", "[14.07711544,  10.80274635,  5.81212135]"]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")